In [ ]:
# pip install langchain
# pip install langchain-community # langchain의 일부 document_loaders 기능은 langchain_community 모듈에 있음
# pip install faiss-cpu
# pip install faiss-gpu
# pip install -qU langchain_community faiss-cpu
# pip install pypdf
# pip install sentence_transformers
# pip install rank_bm25

In [ ]:
from langchain_community.document_loaders import PyPDFLoader # 1.로드 (파일에 따라 TextLoader도 가능)
from langchain.text_splitter import RecursiveCharacterTextSplitter # 2.청크
from langchain.embeddings import HuggingFaceEmbeddings # 3. 임베딩 모델 
from langchain.vectorstores import FAISS # 3. 벡터 저장
from langchain.retrievers import BM25Retriever, EnsembleRetriever # 4. 검색 기법
from rank_bm25 import BM25Okapi
import numpy as np

### 1. dense retrieval 사용

In [8]:
# 1. 문서로딩
loader = PyPDFLoader("report.pdf")
documents = loader.load()

# 2. 문서를 적절한 크기의 조각으로 청크 (split)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100) # # 1000자 크기의 청크로 나누는 예
chunks = text_splitter.split_documents(documents)

# 3. HuggingFaceBgeEmbeddings 사용하여 벡터 임베딩 생성
embedding = HuggingFaceBgeEmbeddings() #model_name="BAAI/bge-small-en-v1.5" 지정 가능 

# 3. FAISS 벡터스토어 생성
vectorstore = FAISS.from_documents(documents=chunks, embedding=embedding)

# 4. 질의 실행 및 top-k 문서 검색
query = "승객수 데이터는 뭐를 사용했어?"
top_k = 3  # 원하는 top-k 문서 개수 설정
results = vectorstore.similarity_search(query, k=top_k)
# retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": top_k})
# search_result = retriever.get_relevant_documents(query)


# 결과 출력
print("Top-k Documents:")
for doc in results:
    print(doc.page_content)  # 각 문서의 내용 출력

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3553: LangChainDeprecationWarning: Default values for HuggingFaceBgeEmbeddings.model_name were deprecated in LangChain 0.2.5 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceBgeEmbeddings constructor instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.3k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

Top-k Documents:
8 
  C 데이터 : 388개. 
 A & B 데이터 : 356개. 
 32개 데이터  누락 확인. 
◼ 누락된  정류장의  승/하차 평균 승객 수 보완 
 인근 버스 배차 수를 바탕으로  유사한  정류장을  찾아 평균 승객 수 계산. 
 특수 경우 (투어버스  노선 등), 최소 승객 수 10명 설정. 
 
[그림 3.3] 해결방안 -인근 버스 배차 수를 바탕으로  승/하차 평균 승객 수를 계산 방법 
3.2.2 미세먼지  및 초미세먼지  데이터  선정 
◼ 스마트쉘터의  공기청정기  필요성을  뒷받침하기  위해 사용. 
◼ 서울시  스마트서울  도시데이터  센서(S-DoT)** 의 측정값  사용. 
◼ 하루 10분마다  수집된  데이터를  평균하여  한 달의 대표값  선정. 
 
[그림 3.4] 미세먼지 , 초미세먼지  데이터  대표값  선정 방법
7 
  광고효과 : 유동인구와  관광지와의  거리 데이터  선정. 
◼ 여행객  대상 
 관광지와의  거리: 스마트  쉘터와  관광지  사이의  거리 데이터  선정. 
 근접 지하철역과의  거리: 지하철역과의  거리 데이터  선정. 
◼ 고령 이용객  대상 
 디지털  취약도 : 고령층의  디지털  이용 취약도  데이터  선정. 
◼ 휠체어  이용객  대상 
 경사도 : 휠체어  이용에  영향을  미치는  경사도  데이터  선정 이용객  수 및 우
선순위  선정. 
◼ 이용객  수 특성 활용 
 서울시  버스 정류장의  이용률과  유동인구를  고려. 
3.2 데이터별  변수 도출 
3.2.1 승객 수 데이터  
◼ 설치 우선화를  위한 정류장별  총 이용객  수 데이터  사용. 
◼ 변수 생성: 정류장별  총 이용객  수를 설명할  수 있는 데이터로  변수 생성. 
◼ 사용한  데이터  
 서울시  버스노선별  시간대별  승/하차 인원 정보: 시간대별  승객 수 데이터 . 
 서울시  버스정류소  위치 정보: 정류소의  위치 데이터를  사용. 
 서울시  

문서를 더 작은 청크(Chunks)로 나눔 </br>
- 저장하고 검색할 때 효율적으로 사용할 수 있도록 준비

### 2. sparse retrieval 사용

In [10]:
# 1. PDF 문서 로드
loader = PyPDFLoader("report.pdf")
documents = loader.load()

# 직접 함수화하여 구현하는 방법(bm25)
# # 각 문서를 토큰화
# def tokenize_documents(documents):
#     return [doc.page_content.split(" ") for doc in documents]

# # BM25 모델을 사용하여 top-k 문서 검색 및 점수 계산
# def search_bm25(query, documents, top_k=5):
#     tokenized_corpus = tokenize_documents(documents)
#     bm25 = BM25Okapi(tokenized_corpus)

#     tokenized_query = query.split(" ")
#     scores = bm25.get_scores(tokenized_query)  # 각 문서에 대한 점수 계산

#     # 점수를 기반으로 top-k 문서 선택
#     top_k_indices = np.argsort(scores)[::-1][:top_k]  # 점수 내림차순으로 top-k 인덱스 선택
#     top_k_docs = [documents[i] for i in top_k_indices]

#     return top_k_docs, scores  # top-k 문서와 점수 반환
# # 질의
# query = "승객수 데이터는 뭐를 사용했어?"

# # BM25를 사용하여 top-3 문서 검색 및 점수 계산
# top_k_docs, scores = search_bm25(query, documents, top_k=3)

bm25_retriever = BM25Retriever.from_documents(chunks)
bm25_retriever.k = 2
results = bm25_retriever.get_relevant_documents(query) # BM25Retriever를 사용하여 query에 맞는 문서 검색

# 검색 결과 출력
for i, doc in enumerate(results):
    print(f"Rank: {i+1}, Content: {doc.page_content}")

Top-k Documents:
Document 1:
18 
 ◼ 광고효과 /디지털취약도 , 관광지 /지하철역 , 승차/하차/출근/퇴근 승객수의  3가지 
쌍의 변수들  주성분  도출. 
◼ 3쌍의 변수들의  첫번째  주성분 (pc1) 설명력  : 광고효과 /디지털  취약도  0.945, 관
광지/지하철역  0.936, 승차/하차/출근/퇴근 승객수  0.9267  
◼ 모든 첫번째  주성분 (pc1)의 설명력이  0.9이상이므로  첫번째  주성분  만을 이용하
여 8개의 변수를  3개의 주성분으로  축소 진행. 
◼ 최종적으로  주성분  분석을  진행하여  12개의 변수로 7개의 최적 변수 도출 
 
[그림 4.12] 주성분  개수에  따른 변수 설명력  
4.4 엔트로피  가중치  기반 순위 결정 
◼ 각 특성별  가중치  결과. 
◼ 중요도  지수 기반 최종 설치 우선 순위 결정. 
     
[표 4.3] 각 특성의  가중치  값 구하기      [표 4.4] 최종 설치 우선 순위 (상위 30개) 

Score: 2.502189108874064
Document 2:
7 
  광고효과 : 유동인구와  관광지와의  거리 데이터  선정. 
◼ 여행객  대상 
 관광지와의  거리: 스마트  쉘터와  관광지  사이의  거리 데이터  선정. 
 근접 지하철역과의  거리: 지하철역과의  거리 데이터  선정. 
◼ 고령 이용객  대상 
 디지털  취약도 : 고령층의  디지털  이용 취약도  데이터  선정. 
◼ 휠체어  이용객  대상 
 경사도 : 휠체어  이용에  영향을  미치는  경사도  데이터  선정 이용객  수 및 우
선순위  선정. 
◼ 이용객  수 특성 활용 
 서울시  버스 정류장의  이용률과  유동인구를  고려. 
3.2 데이터별  변수 도출 
3.2.1 승객 수 데이터  
◼ 설치 우선화를  위한 정류장별  총 이용객  수 데이터  사용. 
◼ 변수 생성: 정류장별  총 이용객  수를 설명할  수 있는 데이터로  변수 생성. 
◼ 사용한  데이

### 3. 앙상블 사용

In [ ]:
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, retriever], weights=[0.5, 0.5])